In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0028'
MAX_BIAS = 1
start_idx = 1550 if kitti_seq == '0028' else 0

In [ ]:
# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

Plot spoofed trajectories

In [ ]:
TRAJLEN = 1900
gps_spoofing_biases = np.zeros(TRAJLEN)  
attack_start_idx = 1000
gps_spoofing_biases[attack_start_idx:] = MAX_BIAS * np.linspace(0, 1, TRAJLEN-attack_start_idx)  # Ramping attack

spoof_pos_1 = gt_enu[:TRAJLEN].copy()
spoof_pos_1[:,0] += -20 * gps_spoofing_biases
spoof_pos_2 = gt_enu[:TRAJLEN].copy()
spoof_pos_2[:,0] += -50 * gps_spoofing_biases
spoof_pos_3 = gt_enu[:TRAJLEN].copy()
spoof_pos_3[:,0] += -100 * gps_spoofing_biases

gt_traj = go.Scatter(x=gt_enu[:TRAJLEN,0], y=gt_enu[:TRAJLEN,1], hovertext=np.arange(TRAJLEN), name='Ground-truth', line=dict(color='black'))
spoof_traj_1 = go.Scatter(x=spoof_pos_1[:,0], y=spoof_pos_1[:,1], hovertext=np.arange(TRAJLEN), name='0.2 m/s spoofed', line=dict(color='red', dash='dot'))
spoof_traj_2 = go.Scatter(x=spoof_pos_2[:,0], y=spoof_pos_2[:,1], hovertext=np.arange(TRAJLEN), name='0.5 m/s spoofed', line=dict(color='red', dash='dash'))
spoof_traj_3 = go.Scatter(x=spoof_pos_3[:,0], y=spoof_pos_3[:,1], hovertext=np.arange(TRAJLEN), name='1.0 m/s spoofed', line=dict(color='red', dash='dashdot'))

start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='black'), showlegend=False)
spoof_start = go.Scatter(x=[gt_enu[attack_start_idx,0]], y=[gt_enu[attack_start_idx,1]], 
            name='Spoofing start', mode='markers', marker=dict(size=10, color='red'), showlegend=True)
fig = go.Figure(data=[gt_traj, spoof_traj_1, spoof_traj_2, spoof_traj_3, start, spoof_start])
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

Plot Lidar odometry, blind FGO, and FGO

In [ ]:
# Lidar odometry
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

In [ ]:
# Load results
run_name = 'fgo_100m_5runs_2023-01-19-0507'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_results = np.load(os.path.join(results_path, results_files[0]))
fgo_positions = fgo_results['positions']

run_name = 'fgo_100m_5runs_blind_2023-01-19-0528'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_blind_results = np.load(os.path.join(results_path, results_files[0]))
fgo_blind_positions = fgo_blind_results['positions']

spoofed_positions = fgo_results['spoofed']

In [ ]:
# Plot
ATTACK_START = 1000
N = 1900
fgo_traj = go.Scatter(x=fgo_positions[:,0], y=fgo_positions[:,1], name='FGO', line=dict(color='blue'))
fgo_blind_traj = go.Scatter(x=fgo_blind_positions[:,0], y=fgo_blind_positions[:,1], name='FGO blind', line=dict(color='orange'))
gt_traj = go.Scatter(x=gt_enu[:N,0], y=gt_enu[:N,1], name='Ground-truth', line=dict(color='black'))
lidar_traj = go.Scatter(x=lidar_positions[:N,0], y=lidar_positions[:N,1], name='Lidar odometry', line=dict(color='green'))
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
plot_data = [fgo_traj, fgo_blind_traj, lidar_traj, gt_traj, start]

spoof_traj = go.Scatter(x=spoofed_positions[:,0], y=spoofed_positions[:,1], 
    name='Spoofed', line=dict(color='red', dash='dash')) 
spoof_start = go.Scatter(x=[spoofed_positions[ATTACK_START,0]], y=[spoofed_positions[ATTACK_START,1]], 
    name='Spoofing start', mode='markers', marker=dict(size=10, color='red'), showlegend=False)
plot_data += [spoof_traj, spoof_start]


# detect = go.Scatter(x=[graph_positions[detect_idx,0]], y=[graph_positions[detect_idx,1]], 
#     name='Detection', mode='markers', hovertext=str(detect_idx), marker=dict(size=10, color='green'), showlegend=True)
# plot_data += [detect]

fig = go.Figure(data=plot_data)
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.02, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

Plot test statistic

In [ ]:
run_name = 'fgo_100m_5runs_blind_2023-01-19-0727'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_blind_results = np.load(os.path.join(results_path, results_files[0]))

In [ ]:
#qs_plot = fgo_results['qs']
qs_plot = fgo_blind_results['qs']
T = fgo_results['threshold']
GPS_RATE = 10
detect_idx = np.argmax(qs_plot > T)

# Plot test statistic and threshold
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=qs_plot, name='Test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=T*np.ones(len(qs_plot)), name='Threshold', line=dict(color='black', dash='dash')))
# Add vertical line at start of spoofing attack
fig.add_trace(go.Scatter(x=[100, 100], y=[min(qs_plot), max(qs_plot)], mode='lines', name='Start of attack', line=dict(color='red', dash='dash')))
# fig.add_shape(type="line", x0=ATTACK_START/GPS_RATE, y0=-20, x1=ATTACK_START/GPS_RATE, y1=200, name='Start of attack', 
#               line=dict(color="red", width=2, dash="dash"), showlegend=True)
fig.add_trace(go.Scatter(x=[detect_idx], y=[qs_plot[detect_idx]], name='Detection', mode='markers', marker=dict(size=10, color='red'), showlegend=True))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.show()

Plot error over time

In [ ]:
kitti_seq = '0027'
fgo_run = 'fgo_100m_5runs_2023-01-19-0313'
fgo_blind_run = 'fgo_100m_5runs_blind_2023-01-19-0331'

In [ ]:
fgo_results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, fgo_run)
fgo_results_files = os.listdir(fgo_results_path)

fgo_blind_results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, fgo_blind_run)
fgo_blind_results_files = os.listdir(fgo_blind_results_path)

start_idx = 1550 if kitti_seq == '0028' else 0
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

n_runs = len(fgo_blind_results_files)
fgo_blind_err_avg = np.zeros(1900)
fgo_err_avg = np.zeros(1900)

for i in range(n_runs):
    fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[i]))
    fgo_blind_results = np.load(os.path.join(fgo_blind_results_path, fgo_blind_results_files[i]))
    fgo_positions = fgo_results['positions']
    fgo_blind_positions = fgo_blind_results['positions']
    
    fgo_blind_err_avg += np.linalg.norm(fgo_blind_positions - gt_enu[:N], axis=1)
    fgo_err_avg += np.linalg.norm(fgo_positions - gt_enu[:N], axis=1)


lidar_err = np.linalg.norm(lidar_positions[:N] - gt_enu[:N], axis=1)
fgo_blind_err_avg /= n_runs
fgo_err_avg /= n_runs

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(lidar_err)), y=lidar_err, name='Lidar odometry'))
fig.add_trace(go.Scatter(x=np.arange(len(fgo_blind_err_avg)), y=fgo_blind_err_avg, name='FGO blind'))
fig.add_trace(go.Scatter(x=np.arange(len(fgo_err_avg)), y=fgo_err_avg, name='FGO'))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='L2 norm error [m]')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18), title=f'{kitti_seq} - 100m - 5 runs')

Monte carlo trajectory plots

In [ ]:
kitti_seq = '0028'
run_name = 'fgo_100m_5runs_2023-01-19-0507'

In [ ]:
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)

start_idx = 1550 if kitti_seq == '0028' else 0
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

N_SHIFT = 10

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))
    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]
    if MAX_BIAS != 0:
        spoofed_positions = results['spoofed']
    else: 
        spoofed_positions = None

    qs = results['qs']
    threshold = results['threshold']
    if any(qs > threshold):
        detect_idx = N_SHIFT * np.argmax(qs > threshold)
    else:
        detect_idx = None

    plot_trajectories(gt_enu, graph_positions, spoofed_positions, detect_idx)

Monte carlo errors

In [ ]:
from plotly.subplots import make_subplots

def plot_errors(fig, gt_enu, graph_positions):
    traj_len = len(gt_enu)
    
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,0] - gt_enu[:,0], name='x error'), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,1] - gt_enu[:,1], name='y error'), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,2] - gt_enu[:,2], name='z error'), row=3, col=1)

In [ ]:
# Error envelope
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))
    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]
    plot_errors(fig, gt_enu, graph_positions)

fig.update_layout(width=1200, height=700)
fig.update_layout(font=dict(size=15))
fig.show()

Error statistics

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
# Nominal

false_alarms = 0
total_trials = 0
trajectory_false_alarms = 0

ss_start = 0  # Start of steady state
xyz_errors = []

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))

    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]

    xyz_errors.append(gt_enu[ss_start:] - graph_positions[ss_start:])

    # False alarm
    qs = results['qs']
    threshold = results['threshold']
    false_alarms += np.sum(qs > threshold)
    total_trials += len(qs)
    if any(qs > threshold):
        trajectory_false_alarms += 1


print(f'Per trial false alarm rate: {false_alarms} of {total_trials}')
print(f'Per trajectory false alarm rate: {trajectory_false_alarms} of {len(results_files)}')

In [ ]:
xyz_errors = np.vstack(xyz_errors)

In [ ]:
print("mean: ", np.mean(xyz_errors, axis=0))
print("std: ", np.std(xyz_errors, axis=0))